In [1]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
from sklearn.metrics import r2_score
import pickle
from tqdm import tqdm
import csv

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [12]:
# import the data
import os

path = '../preprocessed_data/individual_targets/'

R2 = np.empty((2,8))
KF_models = []
# User 
lag=0 #What time bin of spikes should be used relative to the output
#(lag=-1 means use the spikes 1 bin before the output)

col = 0
for filename in os.listdir(path):
    if filename.endswith('.pickle'):
        with open(os.path.join(path, filename),'rb') as f:
            M1, PMd, M1_PMd,pos_binned,vels_binned=pickle.load(f,encoding='latin1') #If using python 3
            
            # group neural data and kinematics
            neural_data_combined = [M1]
            for (i, neural_data) in enumerate(neural_data_combined):

                X_kf = neural_data
                #The final output covariates include position, velocity, and acceleration
                y_kf=np.concatenate((pos_binned,vels_binned),axis=1)


                # Take into account lag (if it applies to the analysis)
                num_examples_kf=X_kf.shape[0]

                #Re-align data to take lag into account
                if lag<0:
                    y_kf=y_kf[-lag:,:]
                    X_kf=X_kf[0:num_examples+lag,:]
                if lag>0:
                    y_kf=y_kf[0:num_examples-lag,:]
                    X_kf=X_kf[lag:num_examples,:]
    
    
                # Split into training / testing / validation sets
                #Set what part of data should be part of the training/testing/validation sets
                training_range=[0.15, 1]
                testing_range=[0.7, 0.85]
                valid_range=[0,0.15]

                # Split data:
                num_examples_kf=X_kf.shape[0]

                ##Note that each range has a buffer of 1 bin at the beginning and end
                #This makes it so that the different sets don't include overlapping data
                training_set=np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
                testing_set=np.arange(int(np.round(testing_range[0]*num_examples_kf))+1,int(np.round(testing_range[1]*num_examples_kf))-1)
                valid_set=np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)

                #Get training data
                X_kf_train=X_kf[training_set,:]
                y_kf_train=y_kf[training_set,:]

                #Get testing data
                X_kf_test=X_kf[testing_set,:]
                y_kf_test=y_kf[testing_set,:]

                #Get validation data
                X_kf_valid=X_kf[valid_set,:]
                y_kf_valid=y_kf[valid_set,:]

                #  Preprocessing: Normalization and zero-centering
                #Z-score inputs 
                X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
                X_kf_train_std=np.nanstd(X_kf_train,axis=0)
                X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
                X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
                X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

                #Zero-center outputs
                y_kf_train_mean=np.mean(y_kf_train,axis=0)
                y_kf_train=y_kf_train-y_kf_train_mean
                y_kf_test=y_kf_test-y_kf_train_mean
                y_kf_valid=y_kf_valid-y_kf_train_mean

                # Run decoder
                # Declare model
                model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter that is set to the default in this example (see ReadMe)

                #Fit model
                model_kf.fit(X_kf_train,y_kf_train)

                #Get predictions
                y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

                R2_pos_vw = np.array([r2_score(y_kf_valid[:,0:2],y_valid_predicted_kf[:,0:2], multioutput='variance_weighted')])
                R2_vel_vw = R2_vw_2 = np.array([r2_score(y_kf_valid[:,2:4],y_valid_predicted_kf[:,2:4], multioutput='variance_weighted')])
                # Save the R2 value for a given neural data and kinematics
                R2[0,col] = R2_pos_vw
                R2[1,col] = R2_vel_vw

                # store the model - maybe needed later
                KF_models.append(model_kf)
            col = col +1

In [13]:
R2_mean = np.average(R2,axis=1)
print(R2_mean)

[0.36440311 0.44742724]


In [5]:
# get average of R2 values for each kinematic
R2_mean = np.average(R2,axis=1)
R2_std = np.std(R2,axis=1)

# Export the data into a csv file for further analysis
csv_rowlist = [['R^2 values','Kalman Filter', 'M1 data'], ['Target','1','2','3','4','5','6','7','8','Average', 'Std. dev.'], ['position',R2[0,0],R2[0,1],R2[0,2],R2[0,3],R2[0,4],R2[0,5],R2[0,6],R2[0,7],R2_mean[0],R2_std[0]], ['velocity',R2[1,0],R2[1,1],R2[1,2],R2[1,3],R2[1,4],R2[1,5],R2[1,6],R2[1,7],R2_mean[1],R2_std[1]]]

export_path = '../Results/Ind_tar/KF_raw_RS_tar.csv'
with open(export_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_rowlist)

In [6]:
print(R2)

[[ 0.55620767  0.57202969  0.50066018  0.32536452  0.34179489  0.23041525
   0.26839182  0.20073228]
 [ 0.34756191  0.3820538   0.58136809  0.49652826  0.54937019  0.16222823
  -0.11113971  0.60134446]]
